In [1]:
import os
import cv2
import shutil
import deeplabcut
import numpy as np
import pandas as pd
import subprocess

2023-03-31 17:27:21.139476: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-31 17:27:21.242186: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-03-31 17:27:21.563372: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/yramakrishna/anaconda3/lib/python3.9/site-packages/cv2/../../lib64:/usr/local/cuda/lib64:
2023-03-31 17:27:21.563418: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic l

Loading DLC 2.3.1...


In [2]:
start_file  = '/home/yramakrishna/DeepLabCut/conda-environments/iggJos_20211220_092035_00049.avi'

In [3]:
subprocess.run(['ffmpeg', '-i', start_file, '-c:v', 'libx264', '-preset', 'slow', '-crf', '22', '-c:a', 'copy', 'outputu.mp4'])

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

CompletedProcess(args=['ffmpeg', '-i', '/home/yramakrishna/DeepLabCut/conda-environments/iggJos_20211220_092035_00049.avi', '-c:v', 'libx264', '-preset', 'slow', '-crf', '22', '-c:a', 'copy', 'outputu.mp4'], returncode=0)

In [4]:
#dsv = deeplabcut.DownSampleVideo(start_file, width=640, height=480)

In [5]:
start_file2  = '/home/yramakrishna/DeepLabCut/conda-environments/outputu.mp4'

In [6]:
#cropping video and Gamma correction

# Load the video file
video = cv2.VideoCapture(start_file2)

# Get the frame rate and total number of frames
fps = int(video.get(cv2.CAP_PROP_FPS))
total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))

#*********************************************

# Set the start and end points to crop out
start_sec = 0  # Crop out the first x seconds
end_sec = 0 # Crop out the last y seconds

#Set gamma correction value
gamma = 3

#**********************************************

start_frame = int(start_sec * fps)
end_frame = int((total_frames / fps) - end_sec) * fps

# Set the video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('CropGr.mp4', fourcc, fps, (int(video.get(cv2.CAP_PROP_FRAME_WIDTH)), int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))))

# Set the current frame number to the start frame
video.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

# Loop through the frames and write them to the output video
for i in range(start_frame, end_frame):
    ret, frame = video.read()
    if not ret:
        break
    
    # Normalize the pixel values to the range [0, 1]
    frame_normalized = frame.astype(np.float32) / 255.0

    # Apply gamma correction
    frame_corrected = np.power(frame_normalized, 1/gamma)

    # Scale the pixel values back to the range [0, 255]
    frame_scaled = (frame_corrected * 255.0).astype(np.uint8)

    out.write(frame_scaled)

# Release the video objects
video.release()
out.release()
cv2.destroyAllWindows()

In [7]:
fps

1

In [8]:
os.mkdir("final_video")

In [9]:
shutil.move('CropGr.mp4', 'final_video/CropGr.mp4')

'final_video/CropGr.mp4'

In [10]:
os.chdir('final_video')

In [11]:

file = 'CropGr.mp4'
bodyparts = ['RightEye_Outer', 'RightEye_Top', 'RightEye_Bottom', 'RightEye_Inner', 'RightEye_Pupil', 'LeftEye_Outer', 'LeftEye_Top', 'LeftEye_Bottom', 'LeftEye_Inner', 'LeftEye_Pupil',
             'OutlineTop_Mid','RightNostrils_Top', 'RightNostrils_Bottom','LeftNostrils_Top', 'LeftNostrils_Bottom']
videotype = os.path.splitext(file)[-1].lstrip('.')  # or MOV, or avi, whatever you uploaded!
video_down = file

model_options = deeplabcut.create_project.modelzoo.Modeloptions
model_selection = 'primate_face'


In [12]:
project_name = 'DLC_GazeXBI'
your_name = 'anc'

config_path, train_config_path = deeplabcut.create_pretrained_project(
    project_name,
    your_name,
    video_down,
    videotype=videotype,
    model=model_selection,
    analyzevideo=True,
    createlabeledvideo=False,
    copy_videos=True,
)


Created "/home/yramakrishna/DeepLabCut/conda-environments/final_video/DLC_GazeXBI-anc-2023-03-31/videos"
Created "/home/yramakrishna/DeepLabCut/conda-environments/final_video/DLC_GazeXBI-anc-2023-03-31/labeled-data"
Created "/home/yramakrishna/DeepLabCut/conda-environments/final_video/DLC_GazeXBI-anc-2023-03-31/training-datasets"
Created "/home/yramakrishna/DeepLabCut/conda-environments/final_video/DLC_GazeXBI-anc-2023-03-31/dlc-models"
1  videos from the directory . were added to the project.
Copying the videos
/home/yramakrishna/DeepLabCut/conda-environments/final_video/DLC_GazeXBI-anc-2023-03-31/videos/CropGr.mp4
Generated "/home/yramakrishna/DeepLabCut/conda-environments/final_video/DLC_GazeXBI-anc-2023-03-31/config.yaml"

A new project with name DLC_GazeXBI-anc-2023-03-31 is created at /home/yramakrishna/DeepLabCut/conda-environments/final_video and a configurable file (config.yaml) is stored there. Change the parameters in this file to adapt to your project's needs.
 Once you hav

/home/yramakrishna/DeepLabCut/conda-environments/final_video/DLC_GazeXBI-anc-2023-03-31/dlc-models/iteration-0/DLC_GazeXBIMar31-trainset95shuffle1/train/pose_cfg.yaml
Analyzing video...
Using snapshot-1030000 for model /home/yramakrishna/DeepLabCut/conda-environments/final_video/DLC_GazeXBI-anc-2023-03-31/dlc-models/iteration-0/DLC_GazeXBIMar31-trainset95shuffle1


/home/yramakrishna/anaconda3/lib/python3.9/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '
2023-03-31 17:27:31.923856: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-31 17:27:31.937331: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-31 17:27:31.937502: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-31 17:27:31.937830: I tensorflow/core/platfo

Analyzing all the videos in the directory...
Starting to analyze %  /home/yramakrishna/DeepLabCut/conda-environments/final_video/DLC_GazeXBI-anc-2023-03-31/videos/CropGr.mp4
Loading  /home/yramakrishna/DeepLabCut/conda-environments/final_video/DLC_GazeXBI-anc-2023-03-31/videos/CropGr.mp4
Duration of video [s]:  1.0 , recorded with  1.0 fps!
Overall # of frames:  1  found with (before cropping) frame dimensions:  1280 960
Starting to extract posture


  0%|                                                     | 0/1 [00:00<?, ?it/s]2023-03-31 17:27:33.533437: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8800
2023-03-31 17:27:33.912510: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
100%|█████████████████████████████████████████████| 1/1 [00:03<00:00,  3.55s/it]

Saving results in /home/yramakrishna/DeepLabCut/conda-environments/final_video/DLC_GazeXBI-anc-2023-03-31/videos...
Saving csv poses!
The videos are analyzed. Now your research can truly start! 
 You can create labeled videos with 'create_labeled_video'
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract a few representative outlier frames.


In [13]:
edits = {
    'dotsize': 1.5,  # size of the dots!
    'pcutoff': 0.3,  # the higher, the more conservative the plotting!
}
deeplabcut.auxiliaryfunctions.edit_config(config_path, edits)

project_path = os.path.dirname(config_path)
full_video_path = os.path.join(
    project_path,
    'videos',
    os.path.basename(video_down),
)

# filter predictions (should already be done above ;):
deeplabcut.filterpredictions(config_path, full_video_path, videotype=videotype)

# re-create the video with your edits!
# deeplabcut.CropVideo(config_path, full_video_path, 'crop', 
deeplabcut.create_labeled_video(config_path, full_video_path, videotype=videotype, displayedbodyparts=bodyparts, draw_skeleton = True, filtered=True)


Filtering with median model /home/yramakrishna/DeepLabCut/conda-environments/final_video/DLC_GazeXBI-anc-2023-03-31/videos/CropGr.mp4
Saving filtered csv poses!


/home/yramakrishna/anaconda3/lib/python3.9/site-packages/scipy/signal/_signaltools.py:1545: UserWarning: kernel_size exceeds volume extent: the volume will be zero-padded.
  warnings.warn('kernel_size exceeds volume extent: the volume will be '


Starting to process video: /home/yramakrishna/DeepLabCut/conda-environments/final_video/DLC_GazeXBI-anc-2023-03-31/videos/CropGr.mp4
Loading /home/yramakrishna/DeepLabCut/conda-environments/final_video/DLC_GazeXBI-anc-2023-03-31/videos/CropGr.mp4 and data.
Duration of video [s]: 1.0, recorded with 1.0 fps!
Overall # of frames: 1 with cropped frame dimensions: 1280 960
Generating frames and creating video.


100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 81.23it/s]


[True]

In [5]:
fl_pth = '/home/yramakrishna/DeepLabCut/conda-environments/Codes/final_video/DLC_GazeXBI-anc-2023-03-30/videos/CropGrDLC_resnet50_DLC_GazeXBIMar30shuffle1_1030000_filtered.h5'

alt_full_video_path = '/home/yramakrishna/DeepLabCut/conda-environments/Codes/final_video/DLC_GazeXBI-anc-2023-03-30/videos/CropGrDLC_resnet50_DLC_GazeXBIMar30shuffle1_1030000_filtered_labeled.mp4'

In [6]:
# deeplabcut.analyze_videos(config_path, 'CropGr.mp4', save_as_csv=True, dynamic=(True,.6,30))

# Load tracking results generated by DeepLabCut
tracking_data = pd.read_hdf(fl_pth)

# Define the names of the facial features that you want to extract frames for
feature_names = ['RightEye_Pupil','LeftEye_Pupil', 'NostrilsTop_Centre', 'OutlineTop_Mid']

# Define the threshold for the confidence score of the facial features
confidence_threshold = 0.9

# Load the input video
cap = cv2.VideoCapture(alt_full_video_path)

# Define the output video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4')
out = cv2.VideoWriter('Crpoutput_video.mp4', fourcc, 30, (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))

# Loop through the video frames and extract frames with facial features
frame_number = 0
cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == True:
        # Get the tracking data for the current frame
        frame_data = tracking_data.loc[frame_number]

        # Check if the desired facial features are present in the frame
        feature_present = False
        check = 0
        for feature_name in feature_names:
            if feature_name in frame_data.loc['DLC_resnet50_DLC_GazeXBIMar30shuffle1_1030000'] and frame_data.loc['DLC_resnet50_DLC_GazeXBIMar30shuffle1_1030000'].loc[feature_name].loc['likelihood'] > confidence_threshold:
                check += 1
        if check==4:
            feature_present = True

        # If the desired facial features are present, save the frame to the output video
        if feature_present: #check==4
            cv2.imshow('output', frame)
            out.write(frame)

        # Display the output
        #cv2.imshow('output', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        # Increment the frame number
        frame_number += 1
    else:
        break

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()

FileNotFoundError: File /home/yramakrishna/DeepLabCut/conda-environments/Codes/final_video/DLC_GazeXBI-anc-2023-03-30/videos/CropGrDLC_resnet50_DLC_GazeXBIMar30shuffle1_1030000_filtered.h5 does not exist

In [16]:
# shutil.move('/home/yramakrishna/DeepLabCut/conda-environments/Codes/final_video/CropGrDLC_resnet50_DLC_GazeXBIMar30shuffle1_1030000_filtered.h5', '/home/yramakrishna/DeepLabCut/conda-environments/Codes/CropGrDLC_resnet50_DLC_GazeXBIMar30shuffle1_1030000_filtered.h5')



'/home/yramakrishna/DeepLabCut/conda-environments/Codes/CropGrDLC_resnet50_DLC_GazeXBIMar30shuffle1_1030000_filtered.h5'

In [20]:


# final_fl_pth = '/home/yramakrishna/DeepLabCut/conda-environments/Codes/CropGrDLC_resnet50_DLC_GazeXBIMar30shuffle1_1030000_filtered.h5
# full_video_path

'/home/yramakrishna/DeepLabCut/conda-environments/Codes/final_video/DLC_GazeXBI-anc-2023-03-30/videos/Crpoutput_video.mp4'

In [18]:
# try:
#     shutil.rmtree('/home/yramakrishna/DeepLabCut/conda-environments/final_video')
# except OSError as e:
#     # If it fails, inform the user.
#     print("Error: %s - %s." % (e.filename, e.strerror))

In [31]:
try:
    os.remove('/home/yramakrishna/DeepLabCut/conda-environments/Codes/Ken_Test_Longdownsampled.mp4')
except OSError as e:
    # If it fails, inform the user.
    print("Error: %s - %s." % (e.filename, e.strerror))

In [ ]:
import tkinter as tk
from tkVideoPlayer import TkinterVideo

root = tk.Tk()

videoplayer = TkinterVideo(master=root, scaled=True)
videoplayer.load(final)
videoplayer.pack(expand=True, fill="both")

videoplayer.play() # play the video

root.mainloop()

Exception in thread Thread-7:
Traceback (most recent call last):
  File "/home/yramakrishna/anaconda3/lib/python3.9/threading.py", line 980, in _bootstrap_inner
    self.run()
  File "/home/yramakrishna/anaconda3/lib/python3.9/threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "/home/yramakrishna/anaconda3/lib/python3.9/site-packages/tkVideoPlayer/tkvideoplayer.py", line 106, in _load
    with av.open(path) as self._container:
  File "av/container/core.pyx", line 401, in av.container.core.open
  File "av/container/core.pyx", line 272, in av.container.core.Container.__cinit__
  File "av/container/core.pyx", line 292, in av.container.core.Container.err_check
  File "av/error.pyx", line 336, in av.error.err_check
av.error.FileNotFoundError: [Errno 2] No such file or directory: '/home/yramakrishna/DeepLabCut/conda-environments/CropGrdownsampledDLC_resnet50_DLC_GazeXBIMar29shuffle1_1030000_filtered_labeled.mp4'


In [ ]:
# cap = cv2.VideoCapture("/home/yramakrishna/DeepLabCut/conda-environments/CropGrDLC_resnet50_DLC_GazeXBIMar20shuffle1_1030000_filtered_labeled.mp4")
# ret, frame = cap.read()
# while(1):
#    ret, frame = cap.read()
#    cv2.imshow('frame',frame)
#    if cv2.waitKey(1) & 0xFF == ord('q') or ret==False :
#        cap.release()
#        cv2.destroyAllWindows()
#        break
#    cv2.imshow('frame',frame)

In [ ]:
# from moviepy.editor import *
# import pygame
 
# clip = VideoFileClip('/home/yramakrishna/DeepLabCut/conda-environments/CropGrDLC_resnet50_DLC_GazeXBIMar20shuffle1_1030000_filtered_labeled.mp4')
# clip.preview()
 
# pygame.quit()

In [ ]:
# from tkinter import *
# from tkvideo import tkvideo

# root = Tk()
# my_label = Label(root)
# my_label.pack()
# player = tkvideo("/home/yramakrishna/DeepLabCut/conda-environments/CropGrDLC_resnet50_DLC_GazeXBIMar20shuffle1_1030000_filtered_labeled.mp4", my_label, loop = 1, size = (1280,720))
# player.play()

# root.mainloop()

In [ ]:
# dlc_model = config_path
# dlc_config = deeplabcut.load_config(config_file)
# dlc_model = deeplabcut.getModel(dlc_config)


# # Load the video file
# cap = cv2.VideoCapture(video_down)

# fps = int(cap.get(cv2.CAP_PROP_FPS))

# fourcc = cv2.VideoWriter_fourcc(*'mp4v')
# out = cv2.VideoWriter('CropMFace.mp4', fourcc, fps, (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))

# # Loop over each frame in the video
# frame_count = 0
# video.set(cv2.CAP_PROP_POS_FRAMES, 0)

# while(cap.isOpened()):
#     ret, frame = cap.read()

#     # Break the loop when the video ends
#     if not ret:
#         break

#     # Detect the monkey face using the DeepLabCut model
#     dlc_coordinates, _ = dlc_model.predict(frame, tracking=True)
#     monkey_face_coordinates = dlc_coordinates['monkey_face']

#     # Check if monkey face is detected in the frame
#     if monkey_face_coordinates.any():
#         out.write(frame_scaled)
#         # Get the bounding box coordinates of the monkey face
#         x, y, w, h = cv2.boundingRect(monkey_face_coordinates.astype(int))

#         # Crop out the monkey face and save it to a new file
#         monkey_face = frame[y:y+h, x:x+w]
#         cv2.imwrite(f'monkey_face_{frame_count}.jpg', monkey_face)
#         frame_count += 1


#     # Break the loop if 'q' is pressed
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# # Release the video capture and close all windows
# out.release()
# cap.release()
# cv2.destroyAllWindows()


In [ ]:

# import cv2
# import numpy as np
# import deeplabcut

# # Load the DeepLabCut model
# config_file = 'path/to/config.yaml'
# dlc_model = deeplabcut.load_model(config_file)

# # Load the video file
# cap = cv2.VideoCapture('path/to/video')

# # Loop over each frame in the video
# frame_count = 0
# while(cap.isOpened()):
#     ret, frame = cap.read()
    
#     # Break the loop when the video ends
#     if not ret:
#         break
    
#     # Detect the monkey face using the DeepLabCut model
#     dlc_coordinates, _ = dlc_model.predict(frame, tracking=True)
#     monkey_face_coordinates = dlc_coordinates['monkey_face']
    
#     # Check if monkey face is detected in the frame
#     if monkey_face_coordinates.any():
#         # Get the bounding box coordinates of the monkey face
#         x, y, w, h = cv2.boundingRect(monkey_face_coordinates.astype(int))
        
#         # Crop out the monkey face and save it to a new file
#         monkey_face = frame[y:y+h, x:x+w]
#         cv2.imwrite(f'monkey_face_{frame_count}.jpg', monkey_face)
#         frame_count += 1
    
#     # Break the loop if 'q' is pressed
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# # Release the video capture and close all windows
# cap.release()
# cv2.destroyAllWindows()
